In [50]:
from keras.models import load_model
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate,Dropout
from keras.models import Model
from keras.utils import CustomObjectScope
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import h5py
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
import csv
import time
import pandas as pd
%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
for layer in FRmodel.layers:
    layer.trainable=False

In [4]:
FRmodel.get_layer('dense_layer').trainable=True
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [7]:
# GRADED FUNCTION: triplet_loss

def triplet_loss(y_true, y_pred, alpha = 0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
    basic_loss = pos_dist - neg_dist + alpha
    
    loss = tf.reduce_sum(tf.maximum(basic_loss,0))
    
    return loss

In [8]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [33]:
# x = FRmodel.get_layer('dense_layer').output
# merge_one = Dense(128, activation='relu', name='last_layer')(x)
# merge_one = Dropout(0.3)(merge_one)
# merge_one = Lambda(lambda  x: K.l2_normalize(x,axis=1))(merge_one)
# FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
os.listdir('lfw/Bryan_Chui')[0]t=time.time()

'Bryan_Chui_0001.jpg'

In [6]:
# model = Model(inputs=FRmodel.input,outputs=merge_one)

In [49]:
t_tot=0
with open('database.csv', 'w', newline='') as csvfile,open('test.csv', 'w', newline='') as csvfile2:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter2 = csv.writer(csvfile2, delimiter=',')
    for num_folder,folder in enumerate(os.listdir('lfw/')):
        t0=time.time()
        for file_num,file in enumerate(os.listdir('lfw/'+folder)):
            file_ad = 'lfw/'+folder+'/'+file
            img_data=img_to_encoding2(file_ad,FRmodel)
            img_data=np.ndarray.tolist(img_to_encoding2(file_ad,FRmodel))       
            img_data=img_data[0]
            img_data.append(folder)
            
            if file_num%2==0:
                spamwriter.writerow(img_data)
            else:
                spamwriter2.writerow(img_data)
        t=time.time()
        t_tot+=(t-t0)
        if int(num_folder%100)==0:
            print(num_folder,'is completed in ',t_tot)
            t_tot=0
#print(img_data)



/home/lavish/Documents/tensorflow3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


0 is completed in  0.2731924057006836
100 is completed in  53.56308889389038
200 is completed in  107.87709426879883
300 is completed in  45.5391685962677
400 is completed in  52.24723696708679
500 is completed in  61.02775835990906
600 is completed in  49.138195514678955
700 is completed in  53.86868453025818
800 is completed in  74.59131479263306
900 is completed in  56.03698420524597
1000 is completed in  65.25782632827759
1100 is completed in  47.31188082695007
1200 is completed in  51.63595962524414
1300 is completed in  51.30126976966858
1400 is completed in  75.28509640693665
1500 is completed in  69.22555637359619
1600 is completed in  131.5475869178772
1700 is completed in  90.64619088172913
1800 is completed in  64.17503595352173
1900 is completed in  50.97138714790344
2000 is completed in  59.940056562423706
2100 is completed in  65.46990823745728
2200 is completed in  84.2909665107727
2300 is completed in  88.81268954277039
2400 is completed in  67.01244330406189
2500 is co

In [206]:
# FUNCTION: who_is_it

def who_is_it(image_path,person,database, model):   

    encoding = img_to_encoding2(image_path,model)
    
    min_dist = 100
    
    for (name, db_enc) in database.items():
        
        dist = np.linalg.norm(encoding-db_enc)
        
        if dist<min_dist:
            min_dist = dist
            c=0
            for letter in name:
                    if(letter.isdigit()):
                        name=name[0:c-1]
                    c+=1
            identity = name
    
    if min_dist < 0.7 and identity==person:
        output=1
    else:
        output=0
        
    return output,identity
# GRADED FUNCTION: who_is_it

def who_is_it2(person,test,database,databaseNames):   

    
    min_dist = 100
    
    for num_sample,sample in enumerate(database):
        
    
        dist = np.linalg.norm(test-sample)
        #print(dist)
        
        if dist<min_dist:
            min_dist = dist
            identity = databaseNames[num_sample]
        #print(min_dist,identity)
    #print(min_dist,identity,person)    
    if min_dist < 0.7 and identity==person:
        output=1
    else:
        output=0
        
    return output

In [200]:
total=0
acc = 0
test=pd.read_csv('test.csv')
database=pd.read_csv('database.csv')
test=test.reset_index().values
database=database.values
# for folder in os.listdir('lfw/'):
#     ln=len([name for name in os.listdir('lfw'+'/'+folder)])
#     if ln>1:
#         for file in os.listdir('lfw/'+folder):
#             file_ad = 'lfw/'+folder+'/'+file
#             out,identity = who_is_it(file_ad,folder,database,FRmodel)
#             total+=1
#             acc+=out
# print(str(acc*100/total)+'% is total acccuracy')

In [201]:
testNames=test[:,-1]
databaseNames=database[:,-1]
database=database[:,0:-1]
test=test[:,1:-1]

In [209]:
#x=who_is_it2(testNames[2],test[2],database,databaseNames)
print(database.shape)

(8925, 128)


In [212]:
count=0
for i in range(0,8925):
    x=who_is_it2(databaseNames[i],database[i],database,databaseNames)
    count+=x
print('The training accuracy is',count/8925,'%')

The training accuracy is 0.9997759103641457 %
